## Capstone Project : Britties Yoga Snack
 
## PART I 

### Problem and Backgound

"Britties" is a new snack, created and developed for the yoga community. It was created by three students who were studying business at New York University (NYU) and wanted to start their entrepreneurial life. One of them was also a yoga professor, who saw a need of a new snack for the yoga community. The product they created  has the best taste and amount of protein per snack and they claimed that it would be a hit their product. In order validate their idea , the want to launch a pilot in the NY area, but due the the limitation in their budget, they must focus in just one area of NY to do the pilot, so they want to apply their data science knowledge in order to find the proper area . 

### Stakeholders

The main stakeholders would be the NY citizens, the yoga community and the three fellow students

## PART II

### Methodology

In order to make the decision differents factors would need to be considered. The first one would be the borough who will have the most Yoga Studio and in second place which Area in the borough will have more density of yoga places. 

### Working with the data

The data that would be used for the project will be the NY Map with the neighbour´s name, latitude, longitude, among other things,  and the data of the yoga places that will be segmented in Foursquare by the category ID:4bf58dd8d48988d102941735

## PART III 

### Processing the Data

#### 1-Importing the libraries

In [32]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
import geopy

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


#### 2-Importing the data

In [33]:
import urllib.request
url = 'https://cocl.us/new_york_dataset'
filename = 'newyork_data.json'
urllib.request.urlretrieve(url, filename)
print('Data downloaded!')

Data downloaded!


In [34]:
with open('newyork_data.json') as NYDF:
    NYDF = json.load(NYDF)

In [35]:
NYDF

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

#### 3-Transforming the data

In [36]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# Transform dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [37]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [38]:
neighborhoods_data = NYDF['features']

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    


In [39]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [40]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### 4-Visualizing the data in a Map

In [41]:
# Use geopy library to get the latitude and longitude values of New York City.
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [42]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)
folium.TileLayer('cartodbdark_matter').add_to(map_newyork)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#fbffb5',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

#### 5-Finding the Yoga Places with Foursquare

In [43]:
CLIENT_ID = 'XLNOKOC5VOWK2HJFY1MHYXWSJDBKUXGIIX4BJSO2O3BMLPI5' # your Foursquare ID
CLIENT_SECRET = 'HBJIKGWE1XZM0NUMIQE33M5H4JHVAHDXQQUZJ1IAVCZ0XCY1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XLNOKOC5VOWK2HJFY1MHYXWSJDBKUXGIIX4BJSO2O3BMLPI5
CLIENT_SECRET:HBJIKGWE1XZM0NUMIQE33M5H4JHVAHDXQQUZJ1IAVCZ0XCY1


In [44]:
LIMIT = 500 #maximum venues with the current account
boroughs = ["Bronx","Manhattan","Brooklyn","Queens","Staten Island"]
results = {}
for borough in boroughs:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        borough,
        LIMIT,
        "4bf58dd8d48988d102941735") # YOGA STUDIO ID
    results[borough] = requests.get(url).json()

In [45]:
results

{'Bronx': {'meta': {'code': 200, 'requestId': '5ecb85bb40a7ea001b1b263e'},
  'response': {'geocode': {'what': '',
    'where': 'bronx',
    'center': {'lat': 40.84985, 'lng': -73.86641},
    'displayString': 'Bronx, NY, United States',
    'cc': 'US',
    'geometry': {'bounds': {'ne': {'lat': 40.917577, 'lng': -73.74806},
      'sw': {'lat': 40.785743, 'lng': -73.933808}}},
    'slug': 'bronx-new-york',
    'longId': '72057594043038202'},
   'headerLocation': 'Bronx',
   'headerFullLocation': 'Bronx',
   'headerLocationGranularity': 'city',
   'query': 'yoga studio',
   'totalResults': 9,
   'suggestedBounds': {'ne': {'lat': 40.8881664, 'lng': -73.82627965},
    'sw': {'lat': 40.8248416, 'lng': -73.92898335000001}},
   'groups': [{'type': 'Recommended Places',
     'name': 'recommended',
     'items': [{'reasons': {'count': 0,
        'items': [{'summary': 'This spot is popular',
          'type': 'general',
          'reasonName': 'globalInteractionReason'}]},
       'venue': {'id': '

In [46]:
df_yoga={}
for borough in boroughs:
    venues = pd.json_normalize(results[borough]['response']['groups'][0]['items'])
    df_yoga[borough] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_yoga[borough].columns = ['Name', 'Address', 'Lat', 'Lng']

In [47]:
df_yoga

{'Bronx':                               Name                    Address        Lat  \
 0                      Bikram Yoga              5500 Broadway  40.876844   
 1                 One Yoga For All           3264 Johnson Ave  40.885288   
 2                       Excel Yoga             3045 Buhre Ave  40.847806   
 3  Sweet Water Dance & Yoga Studio             876 Gerard Ave  40.827720   
 4             Zen-Fit Rehab Studio             410 E 173rd St  40.842716   
 5           Bronx Dahn Yoga Center     1805 Williamsbridge Rd  40.850340   
 6                    B-Well Studio     2125 Williamsbridge Rd  40.856036   
 7                      Zambo Aroma       3848 White Plains Rd  40.885104   
 8                      FinerStudio  1710 Bronxdale Ave Apt 2E  40.845791   
 
          Lng  
 0 -73.906204  
 1 -73.909588  
 2 -73.830948  
 3 -73.924315  
 4 -73.903150  
 5 -73.851810  
 6 -73.855978  
 7 -73.861626  
 8 -73.858552  ,
 'Manhattan':                                             

#### 6-Getting the Data

## With this data let's calculate the distribution of the yoga places

In [48]:
maps = {}
for borough in boroughs:
    borough_lat = np.mean([results[borough]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[borough]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    borough_lng = np.mean([results[borough]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[borough]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[borough] = folium.Map(location=[borough_lat, borough_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_yoga[borough]['Lat'], df_yoga[borough]['Lng'], df_yoga[borough]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[borough])  
    print(f"Numbers of Yoga Studio in {borough} = ", results[borough]['response']['totalResults'])
    print("Showing Top 100")

Numbers of Yoga Studio in Bronx =  9
Showing Top 100
Numbers of Yoga Studio in Manhattan =  227
Showing Top 100
Numbers of Yoga Studio in Brooklyn =  189
Showing Top 100
Numbers of Yoga Studio in Queens =  64
Showing Top 100
Numbers of Yoga Studio in Staten Island =  14
Showing Top 100


### It is really clear than Manhattan and Brooklyn are the two bert options, now let´s vizualise it

In [49]:
maps[boroughs[0]] #Bronx

In [50]:
maps[boroughs[1]] #Manhattan

In [51]:
maps[boroughs[2]] #Brooklyn

In [52]:
maps[boroughs[3]] #Queens

In [53]:
maps[boroughs[4]] #Staten Island

#### Although it is really important to have a lot of yoga studio, the density per area will be crucial for the deplacements , due to the low budget that the students have. So we are going to calculate the density per area in each borough

In [26]:
maps2 = {}
for borough in boroughs:
    borough_lat = np.mean([results[borough]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[borough]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    borough_lng = np.mean([results[borough]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[borough]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps2[borough] = folium.Map(location=[borough_lat, borough_lng], zoom_start=10)
    venues_mean_coor = [df_yoga[borough]['Lat'].mean(), df_yoga[borough]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_yoga[borough]['Lat'], df_yoga[borough]['Lng'], df_yoga[borough]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            legend_name='borough:'+ borough,
            parse_html=False).add_to(maps2[borough])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="red", weight=1.5, opacity=0.5).add_to(maps2[borough])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        caption=''+borough,
        parse_html=False).add_to(maps2[borough])

    print(borough)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_yoga[borough][['Lat','Lng']].values)))

Bronx
Mean Distance from Mean coordinates
0.03473164580176437
Manhattan
Mean Distance from Mean coordinates
0.027892266753226636
Brooklyn
Mean Distance from Mean coordinates
0.031715273837678366
Queens
Mean Distance from Mean coordinates
0.07369320949347496
Staten Island
Mean Distance from Mean coordinates
0.050034817407765574


### The first density area is Manhattan, followed by Brooklyn. Let´s visualize the distribution of the two mos important boroughs for this project

In [27]:
maps2[boroughs[1]]

In [29]:
maps2[boroughs[2]] #Brooklyn

## Conclusion

#### AFTER EVALUATING THE YOGA PLACES DISTRIBUTION IN NY, ME CAN HIGHLY RECOMMEND TO TRY THE LAUNCH PILOT IN MANHATTAN, BECAUSE IT IS THE PLACE WITH MORE YOGA STUDIOS AND ALSO WITH THE HIGHEST DENSITY. HOWEVER IF THE BUDGET OF THE STUDENTS IS REALLY LIMITED BROOKLYN CAN BE ALSO AN OPTIMAL SOLUTION ACCORDING WITH THE ENTERPRISE , BECAUSE IT HAS A GOOD NUMBER OF YOGA PLACES AND ALSO A HIGH DENSITY WITHIN THE AREA